In [ ]:
Part1) Import the required Python, Pandas, Matplotlib, Seaborn packages



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv('/content/titanic_test.csv')
train.head()

In [ ]:
train.info(verbose=True)
d=train.describe()
d

In [ ]:
dT=d.T
dT.plot.bar(y='count')
plt.title("Bar plot of the count of numeric features",fontsize=17)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


train = pd.read_csv('/content/titanic_train (2).csv')
train.head()

sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='RdBu_r')
sns.pairplot(train)

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')


In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')


In [ ]:
f_class_survived=train.groupby('Pclass')['Survived'].mean()
f_class_survived = pd.DataFrame(f_class_survived)
f_class_survived
f_class_survived.plot.bar(y='Survived')
sns.countplot(x='Survived',data=f_class_survived,palette='rainbow')
plt.title("Fraction of passengers survived by class",fontsize=17)


In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='SibSp',data=train,palette='rainbow')


In [ ]:
import numpy as np


train['Age'] = train['Age'].replace('', np.nan)


train['Age'] = train['Age'].astype(float)


train = train.dropna(subset=['Age'])


plt.figure(figsize=(10, 6))
plt.xlabel("Age of the passengers", fontsize=18)
plt.ylabel("Count", fontsize=18)
plt.title("Age histogram of the passengers", fontsize=22)
train['Age'].hist()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.xlabel("Passenger Class",fontsize=18)
plt.ylabel("Age",fontsize=18)
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')


In [ ]:
f_class_Age=train.groupby('Pclass')['Age'].mean()
f_class_Age = pd.DataFrame(f_class_Age)

f_class_Age.plot.bar(y='Age')
plt.title("Average age of passengers by class",fontsize=17)
plt.ylabel("Age (years)", fontsize=17)
plt.xlabel("Passenger class", fontsize=17)


In [ ]:
a=list(f_class_Age['Age'])

def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]

    if pd.isnull(Age):

        if Pclass == 1:
            return a[0]

        elif Pclass == 2:
            return a[1]

        else:
            return a[2]

    else:
        return Age

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)
d=train.describe()

dT=d.T
dT.plot.bar(y='count')
plt.title("Bar plot of the count of numeric features",fontsize=17)

In [ ]:
train.drop('Cabin',axis=1,inplace=True)
train.dropna(inplace=True)
train.head()

In [ ]:

!pip install sklearn-pandas
import pandas as pd
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
if 'Name' in train.columns:
  mapper = DataFrameMapper([
      ('Name', LabelEncoder())
  ])

  train = mapper.fit_transform(train.copy())

In [ ]:
!pip install sklearn-pandas
import pandas as pd
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder

# Convert the NumPy array 'train' back to a Pandas DataFrame
train = pd.DataFrame(train, columns=['Name'])

if 'Name' in train.columns:
  mapper = DataFrameMapper([
      ('Name', LabelEncoder())
  ])

  # Assign the result of the transformation to a new variable to avoid overwriting
  train_transformed = mapper.fit_transform(train.copy())

  # Convert the NumPy array 'train_transformed' to a Pandas DataFrame
  train = pd.DataFrame(train_transformed, columns=['Name'])

In [ ]:
nsimu=101
class_report = [0]*nsimu
f1=[0]*nsimu
random_init =[0]*nsimu

# Ensure the DataFrame has a mix of classes for 'Survived'
train = pd.DataFrame({'Survived': [0, 1, 0, 1, 0, 1],  # Added one more row with 'Survived' as 1
                   'Name': [103, 45, 678, 234, 987, 123]})

for i in range(1,nsimu):
        X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1),
                                                    train['Survived'], test_size=0.3,
                                                    random_state=i+100)

        # Check if y_train has more than one class
        if len(y_train.unique()) > 1:
          logmodel =(LogisticRegression(C=1,tol=1e-5, max_iter=1000,n_jobs=4))
          logmodel.fit(X_train,y_train)
          predictions = logmodel.predict(X_test)
          class_report[i] = classification_report(y_test,predictions)
          l=class_report[i].split()
          f1[i] = l[len(l)-2]
          random_init[i]=i+100
        else:
          print(f"Iteration {i}: y_train contains only one class. Skipping this iteration.")

plt.plot(random_init[1:len(random_init)-2],f1[1:len(f1)-2])
plt.title("F1-score vs. random initialization seed",fontsize=20)
plt.xlabel("Random initialization seed",fontsize=17)
plt.ylabel("F1-score on test data",fontsize=17)
plt.show()